# Assingment 10: Twitter Sentiment Classification

We work on the Kaggle challenge on [Tweet Sentiments](https://www.kaggle.com/c/tweet-sentiment-extraction/overview).

* Task: classify tweets into positive, neutral or negative sentiments.

* Method: train and apply [gensim](https://radimrehurek.com/gensim/auto_examples/index.html#documentation) [doc2vec](https://colab.research.google.com/github/keuperj/DataMining_21/blob/main/Week_10/gensim_doc2vec.ipynb) to encode the tweetes. Then train a simple [RandomForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). 


In [1]:
#update gensim
!pip install --upgrade gensim

     |████████████████████████████████| 24.1 MB 2.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
#get the data
!git clone https://github.com/keuperj/DATA.git

Cloning into 'DATA'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 57 (delta 14), reused 50 (delta 7), pack-reused 0
Unpacking objects: 100% (57/57), done.
Checking out files: 100% (34/34), done.


In [3]:
import pandas as pd

In [4]:
train = pd.read_csv('DATA/twitter_train.csv' , encoding = "ISO-8859-1")
test = pd.read_csv('DATA/twitter_test.csv' , encoding = "ISO-8859-1")

In [5]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [6]:
import gensim

In [7]:
#prepare data
y_train = train['sentiment']
y_test = test[clf.score(train_vec,y_train)'sentiment']
x_train = train['text']
x_test = test['text']

In [9]:
#map lables to numeric values
y_train=y_train.replace(to_replace=['neutral', 'positive','negative'], value=[0, 1, -1])
y_test=y_test.replace(to_replace=['neutral', 'positive','negative'], value=[0, 1, -1])

In [27]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#tokenize and tag the card text
train_corpus = [TaggedDocument(str(doc).split(' '), [i]) 
             for i,doc in enumerate(x_train)]

test_corpus = [TaggedDocument(str(doc).split(' '), [i]) 
             for i,doc in enumerate(x_test)]
#display corpus
train_corpus

[TaggedDocument(words=['', 'I`d', 'have', 'responded,', 'if', 'I', 'were', 'going'], tags=[0]),
 TaggedDocument(words=['', 'Sooo', 'SAD', 'I', 'will', 'miss', 'you', 'here', 'in', 'San', 'Diego!!!'], tags=[1]),
 TaggedDocument(words=['my', 'boss', 'is', 'bullying', 'me...'], tags=[2]),
 TaggedDocument(words=['', 'what', 'interview!', 'leave', 'me', 'alone'], tags=[3]),
 TaggedDocument(words=['', 'Sons', 'of', '****,', 'why', 'couldn`t', 'they', 'put', 'them', 'on', 'the', 'releases', 'we', 'already', 'bought'], tags=[4]),
 TaggedDocument(words=['http://www.dothebouncy.com/smf', '-', 'some', 'shameless', 'plugging', 'for', 'the', 'best', 'Rangers', 'forum', 'on', 'earth'], tags=[5]),
 TaggedDocument(words=['2am', 'feedings', 'for', 'the', 'baby', 'are', 'fun', 'when', 'he', 'is', 'all', 'smiles', 'and', 'coos'], tags=[6]),
 TaggedDocument(words=['Soooo', 'high'], tags=[7]),
 TaggedDocument(words=['', 'Both', 'of', 'you'], tags=[8]),
 TaggedDocument(words=['', 'Journey!?', 'Wow...', 'u',

In [28]:
#define word2doc model
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [29]:
#build vocabulary
model.build_vocab(train_corpus)

In [33]:
#check some words
print(f"Word 'dude' appeared {model.wv.get_vecattr('dude', 'count')} times in the training corpus.")

Word 'dude' appeared 36 times in the training corpus.


In [34]:
#actual training
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [36]:
#test doc2vec embedding of training sample
train_corpus[30].words

['I`m',
 'going',
 'home',
 'now.',
 'Have',
 'you',
 'seen',
 'my',
 'new',
 'twitter',
 'design?',
 'Quite....heavenly',
 'isn`****?']

In [41]:
#use model to vectorize training data
print(model.infer_vector(train_corpus[30].words))


50


In [45]:
#get all embeddings
import numpy as np
train_vec = np.zeros((len(train_corpus),50))
test_vec = np.zeros((len(test_corpus),50))

In [47]:
for i in range(len(train_corpus)):
  train_vec[i,:]=model.infer_vector(train_corpus[i].words)

In [48]:
for i in range(len(test_corpus)):
  test_vec[i,:]=model.infer_vector(test_corpus[i].words)

In [49]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
clf = RandomForestClassifier(random_state=0)
clf.fit(train_vec, y_train)

RandomForestClassifier(random_state=0)

In [53]:
clf.score(train_vec,y_train)

1.0

In [54]:
clf.score(test_vec,y_test)

0.48670062252405205